In [121]:


from tabulate import tabulate
import pandas as pd
import numpy as np
import datetime , pytz
import matplotlib.pyplot as plt
import os
import math 
import glob

import re

import quantstats as qs

import sys


from PIL import Image, ImageDraw, ImageFilter


qs.extend_pandas()


#=== Code to get all performance stats from trades file
#===== Currently thinking about
#--- avg profit/loss(%) , Win/Loss ration

#--- avg winner(%) , avg loser(%)
#-- The formula for calculating the Expectancy Ratio is:

#------------Expectancy Ratio = (Total Profit / Number of Winning Trades) / (Total Loss / Number of Losing Trades)
#---Profit factor
#--------Average Holding Period
#--- Maximum Adverse Excursion , MFE

#--- trade frequency

#-- max consecutive wins/losses



#----xlxs or csv file  file format for input




In [122]:
#---File format ----

#---'Symbol' optional 

#--- trade is direction(long/short ,  Buy/Sell)

#--- entry date , entryprice and  exitdate and exit price and qty required


    # 	Symbol	Trade	Entry_Date	Entry_Price	Exit_ date	Exit_ Price	Qty
    # 0	NIFTY14JAN2114250PE.NFO	Short	2021-01-08 14:55:00	83.74200	2021-01-08 15:25:00	74.10	1234
    # 1	NIFTY14JAN2114300PE.NFO	Short	2021-01-08 14:55:00	101.97300	2021-01-08 15:25:00	90.15	1012




In [123]:
def read_trades(loc):
    import os
    
    if os.path.exists(loc):
    
        #--- get the file extension
        file_ext = os.path.splitext(loc)[1]
        
        # Read the file if it's in CSV format
        if file_ext == '.csv':
            df = pd.read_csv(loc)

        # Read the file if it's in Excel format
        elif file_ext in ['.xls', '.xlsx']:
            df = pd.read_excel(loc)

        # Raise an error if the file format is not supported
        else:
            raise ValueError('File format not supported')
            return None
        #-- if file read is not empty or None
        
        if len(df.columns)!=7:
            
            print("\n \n Incorrect file please check")
            print("\n \n File should have only 7 columns: ----- Symbol , Trade , entry price , entry date , exit price , exit date\n \n ")
            raise ValueError('\n File format not correct. Check number of columns')
            return None  
        
        
        if df is not None and not df.empty :
            df.columns = df.columns.str.lower()
            return df
          
        
        
        
    

In [124]:

def convert_datetime(df, column_name , is_index=0):
    
    # convert index to datetime
    formats = [ '%Y%m%d %H%M%S', '%d-%m-%Y', '%Y-%d-%m %H:%M:%S', '%Y-%m-%d %H:%M:%S', '%Y/%m/%d %H:%M:%S' , '%m-%d-%Y %H:%M:%S' , '%Y-%m-%d', '%m%d%Y %H:%M:%S', '%Y/%m/%d' , "%Y-%d-%m" , '%Y-%m-%dT%H:%M:%S.%f']
    for fmt in formats:
        try:
            if is_index ==0:
                df[column_name] = pd.to_datetime(df[column_name], format=fmt, errors='raise')
                return df
            if is_index==1:
                
                df.index = pd.to_datetime(df.index ,format=fmt, errors='raise' )
                return df
                       
             
            break
                    
        except ValueError:
            pass

In [125]:
def preprocessing(df):

    #-- mappings for different kinds of name found in the trade files
    entry_date_map = {'e.date': 'entry_date', 'entrydate': 'entry_date' , 'date':'entry_date'}
    exit_date_map = {'ex. date': 'exit_date', 'exit_ date': 'exit_date' , 'exitdate': 'exit_date'}
    exit_price_map = {'exit_ price': 'exit_price', 'ex.price': 'exit_price' ,'exit':'exit_price' , 'exitprice':'exit_price' ,'ex. price': 'exit_price' }
    entry_price_map = {'price': 'entry_price', 'entry': 'entry_price' ,'entryprice':'entry_price', 'entry price': 'entry_price' }
    qty_map = {'contract':'qty' , 'contracts':'qty' , 'shares':'qty' , 'lots':'qty' , 'quantity':'qty' }
    
    trade_map = {'side':'trade' , 'position':'trade' }
    

    # Rename the column names using the dictionaries
    
    df.rename(columns=entry_date_map, inplace=True)
    df.rename(columns=exit_date_map, inplace=True)
    df.rename(columns=exit_price_map, inplace=True)
    df.rename(columns=entry_price_map, inplace=True)
    df.rename(columns=qty_map, inplace=True)
    df.rename(columns=trade_map, inplace=True)
    
    
    if 'entry_date' in df.columns:
        convert_datetime(df ,'entry_date'  , 0 )      

    if 'exit_date' in df.columns:

        convert_datetime(df ,'exit_date'  , 0 )


    if 'entry_price' in df.columns:

        df['entry_price'] = df['entry_price'].astype('float')

    if 'exit_price' in df.columns:

        df['exit_price'] = df['exit_price'].astype('float') 


    if 'qty' in df.columns:

        df['qty'] = df['qty'].astype('int')
    
    return df
    

In [135]:
def entry_exposure(df):
    
    if 'qty' in df.columns:
        
        df['qty'] = df['qty'].astype('int')
        
        return df['qty']*df['entry_price'] 
    
        
def exit_exposure(df):
    
    if 'qty' in df.columns:
        
        df['qty'] = df['qty'].astype('int')
        
        return df['qty']*df['exit_price'] 
     
def trade_pnl(df):
    
    """
    Finds the %pnl of each trade from the dataframe of trades 
    
    Parameters:
    df (pandas.Series): A pandas df of trades 
    
    
    Returns:
    dataframe: list of pnl of each trade
    
    """
    long_trade = np.array(['BUY', 'buy', 'Long', '1', 'LONG' , 'Long' , 'Buy'])
    short_trade = np.array(['SELL', 'sell', 'SHORT', '-1', 'Short' , 'short' , 'Sell'])
    allowed_values = np.concatenate([long_trade, short_trade])  # Combine allowed values into one array
    
    trade_values = df['trade'].unique()  # Get unique values of 'trade' column
    
    if not np.in1d(trade_values, allowed_values).all():
        raise ValueError('\n \n Invalid trade(position) value found in Trade File. All signals should be long/short or Buy/Sell \n ')
    long_mask = np.isin(df['trade'], long_trade)
    short_mask = np.isin(df['trade'], short_trade)

    
    
#     long_trade = np.array(['BUY', 'buy', 'Long', '1', 'LONG' , 'Long' , 'Buy'])
#     short_trade = np.array(['SELL', 'sell', 'SHORT', '-1', 'Short' , 'short' , 'Sell'])
#     long_mask = np.isin(df['trade'], long_trade)
#     short_mask = np.isin(df['trade'], short_trade)
    return np.where(long_mask, (exit_exposure(df)/entry_exposure(df)) - 1, 
           np.where(short_mask, (entry_exposure(df)-exit_exposure(df))/entry_exposure(df), None))

In [129]:

def max_consecutive_win_loss(df):

    pnl = trade_pnl(df)

    win_streak = 0
    loss_streak = 0
    max_win_streak = 0
    max_loss_streak = 0

    for i in range(len(pnl)):
        if pnl[i] is not None:
            if pnl[i] > 0:
                win_streak += 1
                loss_streak = 0
                if win_streak > max_win_streak:
                    max_win_streak = win_streak
            else:
                loss_streak += 1
                win_streak = 0
                if loss_streak > max_loss_streak:
                    max_loss_streak = loss_streak

    return max_win_streak , max_loss_streak


def win_percent(df):
        
    pnl = trade_pnl(df)
    return np.round(np.count_nonzero(pnl > 0)/len(pnl) , 2)

def loss_percent(df):
    
    pnl = trade_pnl(df)
    return np.round(np.count_nonzero(pnl<=0)/len(pnl) , 2)    

def avg_profit_perc(df):
    
    return np.round(np.mean(trade_pnl(df)) , 3) 


def rolling_avg_profit_perc(df , window=20):
    
    pnl = trade_pnl(df)
    
    rolling_mean = pd.Series(pnl).rolling(window=window).mean()
    
    ema_pnl = pd.Series(pnl).ewm(span=window, adjust=False).mean()
                                                                                                                
    return ema_pnl


def monthly_pnl(df):
    """
    Aggregates all the trades for the month and finds the net PnL  
    
    Parameters:
    df (pandas.Series): A pandas df of trades 
    
    
    Returns:
    dataframe: of monthly cumulative pnl for each month 
    
    """
    
    import copy
    df_copy = copy.deepcopy(df)
    pnl = trade_pnl(df_copy)
    df_copy['daily_pnl'] = pnl
    
    monthly_pnl = df_copy.resample('M', on='entry_date')['daily_pnl'].sum()
    
    return monthly_pnl


def average_monthy_return(df):
    
    monthly_pnlser = monthly_pnl(df)
    
    return np.round(monthly_pnlser.mean() , 2)

def average_monthly_drawdown(df):
    
    """Calculate the average monthly drawdown percentage for the given DataFrame."""
    # Calculate the monthly returns series
    
    monthly_pnlser = monthly_pnl(df)
    monthly_cumulative_pnl = monthly_pnlser.cumsum()
    
    # Calculate the monthly drawdown for the portfolio
    monthly_drawdown = monthly_cumulative_pnl - monthly_cumulative_pnl.cummax()
    
    # Calculate the percentage drawdown for the portfolio
    monthly_percentage_drawdown = monthly_drawdown / monthly_cumulative_pnl.cummax()
    #print(monthly_percentage_drawdown)
    # Calculate the average monthly drawdown percentage
    return monthly_percentage_drawdown.mean()



In [130]:

#  Expectancy Ratio = (Win Rate x Average Win) / (Loss Rate x Average Loss)
#  >1.25 is a good sign
def expectancy_ration(df):
    
    pnl = trade_pnl(df)
    positive_pnl = pnl[pnl > 0]
   
    mean_positive_pnl = np.mean(positive_pnl)   # calculate the mean of positive values
    
    pos_exp = mean_positive_pnl*win_percent(df)
   
    #=== for losers now
    
    negative_pnl = pnl[pnl<=0]
    mean_negative_pnl = np.mean(negative_pnl)
    
    neg_exp = (abs(mean_negative_pnl))*loss_percent(df)
    
    
    return pos_exp/neg_exp



#--- worst trades based on a quantile
def worst_trades(df , quan=0.02):
    
    """
    Finds the worst(losers) of all the trades for the month based on a quantile.
    eg. if you want top 10% of losers set quan=0.1
    
    Parameters:
    df (pandas.Series): A pandas df of trades 
    
    
    Returns:
    dataframe: of monthly cumulative pnl for each month 
    
    """
    import copy
    df_copy = copy.deepcopy(df)
    
    df_copy['pnl'] = trade_pnl(df_copy)
    
    sorted_trades = df_copy.sort_values(by=['pnl'] , ascending=True)
    
    #--- find the quantile threshold required
    
    threshold  = sorted_trades['pnl'].quantile(quan)
    worst_trades = sorted_trades[sorted_trades['pnl'] < threshold]
    
    return worst_trades
    

def avg_holding_period(df ,trading_hours_per_day=6.5):
    
    """
    
    Finds the average holding period of all trades(in minutes) from entry and exit dates
    
    
    Parameters:
    df (pandas.Series): A pandas df of trades 
    
    
    Returns:
    dataframe: of monthly cumulative pnl for each month 
    
    """
    
    
    
    import copy
    df_copy = copy.deepcopy(df)
    
    
    df_copy['business_days'] = df_copy.apply(lambda row: pd.bdate_range(start=row['entry_date'], end=row['exit_date'], freq='B').size-1 if row['entry_date'].normalize() != row['exit_date'].normalize() else 0, axis=1)
    
    hold_min = (trading_hours_per_day * df_copy['business_days']*60)  + (df_copy['exit_date'].dt.hour*60 + df_copy['exit_date'].dt.minute) - (df_copy['entry_date'].dt.hour*60 + df_copy['entry_date'].dt.minute)     
    
    
    return np.mean(hold_min)    


#--- average number of trades per day

def trade_freq(df):
    
    return df.groupby(by =df['entry_date'].dt.date)['trade'].count().mean()
    
    
def exposure(df):
    
    trading_days = df['entry_date'].dt.date.nunique()
    
    return np.round(trading_days/252 , 2) 
    
def cvar(df , alpha=0.95):
    
    """
    Finds the Cvar of all the trades based on the confidence interval as input
    
    eg. if alpha = 0.95 than Cvar tells that worst 5% of cases we have an average loss = Cvar
    
    Parameters:
    df (pandas.Series): A pandas df of trades 
    alpha = 0.95 if you want 5% worst cases
    
    Returns:
    dataframe: of monthly cumulative pnl for each month 
    
    """
    import copy
    df_copy = copy.deepcopy(df)
    
    
    # Calculate the returns for each trade
    df_copy['Returns'] = trade_pnl(df)

    # Calculate the portfolio returns
    portfolio_returns = df_copy['Returns'].sum()

    # Calculate the portfolio VaR at the desired confidence level
    portfolio_var = np.percentile(df_copy['Returns'], 100*(1-alpha))

    # Calculate the portfolio returns that fall below the VaR
    portfolio_losses = df_copy['Returns'][df_copy['Returns'] < portfolio_var]

    # Calculate the CVaR as the average of the losses that fall below the VaR
    portfolio_cvar = np.mean(portfolio_losses)

    #print(f"The CVaR of the trades at {alpha*100}% confidence is {portfolio_cvar:.4f}.")
    
    return portfolio_cvar*100





In [131]:
xx = worst_trades(df , 0.02)

In [132]:
xx

,symbol,trade,entry_date,entry_price,exit_date,exit_price,qty,business_days,Returns,pnl
673,NIFTY10MAR2216500PE.NFO,Short,2022-03-10 11:15:00,4.85,2022-03-10 12:45:00,15.115,7500,0,-2.116495,-2.116495
469,NIFTY28OCT2118100PE.NFO,Short,2021-10-27 13:55:00,10.40,2021-10-27 14:25:00,27.150,7500,0,-1.610577,-1.610577
340,NIFTY15JUL2115550PE.NFO,Short,2021-07-12 10:35:00,10.65,2021-07-12 13:05:00,27.250,7500,0,-1.558685,-1.558685
307,NIFTY10JUN2115600PE.NFO,Short,2021-06-09 09:55:00,12.85,2021-06-09 13:05:00,31.600,7500,0,-1.459144,-1.459144
596,NIFTY20JAN2218100PE.NFO,Short,2022-01-18 09:25:00,19.90,2022-01-18 09:55:00,46.630,4292,0,-1.343216,-1.343216
526,NIFTY16DEC2117400PE.NFO,Short,2021-12-13 09:25:00,39.45,2021-12-13 12:15:00,90.600,2587,0,-1.296578,-1.296578
306,NIFTY10JUN2115550PE.NFO,Short,2021-06-09 09:55:00,8.50,2021-06-09 13:05:00,19.150,7500,0,-1.252941,-1.252941
537,NIFTY23DEC2116400PE.NFO,Short,2021-12-21 13:15:00,13.75,2021-12-21 14:25:00,27.650,7220,0,-1.010909,-1.010909
627,NIFTY10FEB2217100PE.NFO,Short,2022-02-08 12:45:00,49.90,2022-02-08 13:05:00,98.770,1493,0,-0.979359,-0.979359
697,NIFTY24MAR2217050PE.NFO,Short,2022-03-23 11:05:00,21.15,2022-03-23 11:55:00,41.000,3373,0,-0.938534,-0.938534


In [133]:
def create_trade_report(loc):
    
    df = read_trades(loc)
    
    df = preprocessing(df)
   
    
    
    avgpnl = avg_profit_perc(df)
    winp = win_percent(df)
    losp = loss_percent(df)
    monret = average_monthy_return(df)
    mon_dd = average_monthly_drawdown(df)
    tradef = trade_freq(df)
    holp = avg_holding_period(df , trading_hours_per_day=6.5)
    expec = expectancy_ration(df)
    cvarv = cvar(df , alpha=0.95)

        #==== Metrics to display... 

    vars_dict = {'Average Profit Perc per trade': format(avgpnl*100, '.2f') + '%',
                 'Win Percentage' : format(winp*100, '.2f') + '%',
                 'Loss Percentage': format(losp*100, '.2f') + '%',
                 'Avg Holding period(min)' : format(holp, '.2f'), 
                 'Avg Monthly Return': format(monret, '.2f') ,
                 'Avg Monthly Drawdown': format(mon_dd*100, '.2f') + '%',
                 'Avg Trades per Day': format(trade_freq(df), '.2f'), 
                 '% days traded in a year(days)': format(exposure(df)*100, '.2f') + '%', 
                 'Conditional VaR': format(cvarv , '.2f') + '%' }          

    print(vars_dict)

    vars_df = pd.DataFrame(vars_dict.items(), columns=['Variable', 'Value'] )
    vars_df['Value'] = vars_df['Value'].apply(lambda x: f'<b>{x}</b>')
    vars_html = vars_df.to_html(index=False, header=None, border=2, justify='center', escape=False)

    # Add title

    html = f'''
    <html>
        <head>
            <title style="text-align:center;">Strategy Trade Metrics</title>
            <style>
                .container {{
                    width: 80%;
                    margin: 0 auto;
                    padding: 20px;
                }}
                table {{
                    border-collapse: collapse;
                    width: 100%;
                }}
                th, td {{
                    padding: 10px;
                    text-align: left;
                    border: 1px solid #ddd;
                    font-size: 16px;
                }}
                th {{
                    background-color: #4CAF50;
                    color: white;
                }}
                .table-striped tbody tr:nth-of-type(odd) {{
                    background-color: #f1f1f1;
                }}
                .table-hover tbody tr:hover {{
                    background-color: #f5f5f5;
                }}
            </style>
        </head>
        <body>
            <div class="container">
                <h1 style="text-align: center;">Strategy Trade Metrics</h1>
                {vars_html}
            </div>
        </body>
    </html>
    '''

    # Save the HTML report to a file
    with open(r"C:\Users\aakas\Desktop\Aargo\test\tpplots.html", 'w' , encoding='utf-8') as f:
        f.write(html)

    

In [134]:

loc1 = r"C:\Users\aakas\Downloads\Test1.xlsx"

create_trade_report(loc1)

ValueError: Invalid trade value found in DataFrame.

In [ ]:
average_monthly_drawdown(df)

In [49]:
worst_trades(df , 0.02)

                      symbol  trade          entry_date  entry_price  \
673  NIFTY10MAR2216500PE.NFO  Short 2022-03-10 11:15:00         4.85   
469  NIFTY28OCT2118100PE.NFO  Short 2021-10-27 13:55:00        10.40   
340  NIFTY15JUL2115550PE.NFO  Short 2021-07-12 10:35:00        10.65   
307  NIFTY10JUN2115600PE.NFO  Short 2021-06-09 09:55:00        12.85   
596  NIFTY20JAN2218100PE.NFO  Short 2022-01-18 09:25:00        19.90   
526  NIFTY16DEC2117400PE.NFO  Short 2021-12-13 09:25:00        39.45   
306  NIFTY10JUN2115550PE.NFO  Short 2021-06-09 09:55:00         8.50   
537  NIFTY23DEC2116400PE.NFO  Short 2021-12-21 13:15:00        13.75   
627  NIFTY10FEB2217100PE.NFO  Short 2022-02-08 12:45:00        49.90   
697  NIFTY24MAR2217050PE.NFO  Short 2022-03-23 11:05:00        21.15   
77   NIFTY11FEB2114650PE.NFO  Short 2021-02-09 10:25:00        11.00   
430  NIFTY07OCT2117600PE.NFO  Short 2021-10-06 09:25:00         9.70   
625  NIFTY10FEB2216900PE.NFO  Short 2022-02-08 12:35:00        2

In [ ]:
#--- average holding period in minutes


In [58]:
alpha = 0.95



The CVaR of the trades at 95.0% confidence is -0.8826.


In [59]:
# Calculate the portfolio VaR at the desired confidence level
portfolio_var = np.percentile(df['Returns'], 100*(1-alpha))

In [57]:
portfolio_var

-0.9736432960483611